In [1]:
import json 
import phaselocker as pl
import numpy as np 
from bokeh.io import output_notebook, show
import phaselocker.geometry as geom
import phaselocker.sampling as mc 
import phaselocker.visualization as vis


In [2]:
#Load calculated LiAl data
with open('data/calculated_configs.json','r') as f:
    calc_data = json.load(f)
calc_comp = np.array(calc_data['comp'])
calc_corr = np.array(calc_data['corr'])
calc_ef = np.array(calc_data['formation_energy'])

#Load uncalculated data
with open("data/uncalculated_configs.json",'r') as f:
    uncalc_data = json.load(f)
uncalc_comp = np.array(uncalc_data['comp'])
uncalc_corr = np.array(uncalc_data['corr'])

#Combine datasets to make an over-enumerated dataset
all_comp = np.vstack([calc_comp, uncalc_comp])
all_corr = np.vstack([calc_corr, uncalc_corr])

#Check for duplicate row vectors in all_corr
_, idx = np.unique(all_corr, axis=0, return_index=True)
print("Number of duplicate rows in all_corr: ", all_corr.shape[0] - len(idx))
print(idx)

#Remove duplicate rows
all_comp = all_comp[idx,:]
all_corr = all_corr[idx,:]




Number of duplicate rows in all_corr:  17964
[     0 208748 229298 ... 216967 165577      1]


In [3]:
#Find the observed ground state incides
observed_hull = geom.full_hull(compositions=calc_comp, energies=calc_ef)
observed_gs_indices, simplex_indices = geom.lower_hull(observed_hull)
#Enforce the observed ground states
imposed_vertices = observed_gs_indices
imposed_simplices = observed_hull.simplices[simplex_indices]
print(observed_gs_indices)

[  0   1  15  28  49  83 267 441]


In [4]:
#Define useful functions

#Make a gradient descent function to find the cone, given a starting point
def find_cone(starting_point):
    current_point = starting_point
    for step in range(5000):
        predicted_hull = geom.full_hull(all_comp, all_corr @ current_point)
        _, predicted_simplices = geom.lower_hull(predicted_hull)
        predicted_simplices = predicted_hull.simplices[predicted_simplices]
        neg_grad = geom.orderparam_gradient(
            imposed_gs_corr=calc_corr,
            imposed_gs_comp=calc_comp,
            imposed_gs_simplices=imposed_simplices,
            overenum_corr=all_corr,
            overenum_comp = all_comp,
            predicted_simplices=predicted_simplices,
        )
        
        gradmag = np.linalg.norm(neg_grad)
        if gradmag < 1e-13:
            return current_point
        current_point += 0.002 * neg_grad / gradmag
    return np.nan

#A function to make a model perfectly replicate end state configuration formation energies
#(configurations at compositions 0 and 1)
def rectify_endpoints(eci):
    eci[0] = 0
    eci[1] = calc_ef[1] - eci[0] - np.sum(eci[2:])
    return eci


In [ ]:
#Try to find the cone from multiple starting points
#It helps to start from many different points to obtain an in-cone vector that is more cenered within the cone
#Use +- 1 along four axes as starting points

#(Should take about 2 minutes, 5 seconds to run)

starts = []
for i in range(4,6):
    print(i)
    tmp = np.zeros(calc_corr.shape[1])
    tmp[i]=1
    starts.append(tmp)
    tmp[i] = -1
    starts.append(tmp)

final_ecis = []
for start in starts:
    final_ecis.append(find_cone(start))

4
5


In [10]:
#Take the mean of all final ECI vectors
final_eci = np.mean(final_ecis, axis=0)

In [15]:
#Display the formation energies and lower convex hull of the in-cone model
output_notebook()
p = vis.binary_convex_hull_plotter(comp=calc_comp, observed_energies=calc_ef, predicted_energies=calc_corr@final_eci)
show(p)

Loading BokehJS ...

In [21]:
#Fix the end point energies to match observed values
rectified_eci = rectify_endpoints(final_eci)
p = vis.binary_convex_hull_plotter(comp=calc_comp, observed_energies=calc_ef, predicted_energies=calc_corr@rectified_eci)
show(p)